<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#ScratchPad-From-This-Point-On" data-toc-modified-id="ScratchPad-From-This-Point-On-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>ScratchPad From This Point On</a></span></li></ul></li></ul></div>

In [65]:
from esper.prelude import *
import esper.stdlib as stdlib
from esper.rekall import intrvllists_to_result_with_objects
from esper.shot_scale import ShotScale, get_all_frames_with_shot_scale

In [66]:
def to_simple_display(frames):
    """values need to have frame_number, video_id fields"""
    return stdlib.simple_result([{'video': row.video_id,
                                'min_frame': row.frame_number,
                                 'objects':[]} for row in frames], 'Video')

def pose_payload_to_object(pose, video):
    return {
        "id": video,
        'type': 'pose',
        'labeler': 'UNKNOWN',
        'keypoints': pose
    }

def payload_to_objects(payload, video_id):
    result = []
    result += [esper.rekall.bbox_to_result_object(x, video_id) for x in payload.get('bbox', [])]
    result += [pose_payload_to_object(x, video_id) for x in payload.get('pose', [])]
    return result

esper_widget(
    intrvllists_to_result_with_objects(
        get_all_frames_with_shot_scale(123, ShotScale.XL).get_allintervals(), payload_to_objects,
        limit=1000, stride=1))

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['All faces', 'def all_faces():\n    from query.models i…

## ScratchPad From This Point On

In [238]:
#list(Video.objects.filter(name="the godfather"))
#POSE_ID=5010
#print(Pose.objects.filter(pk=POSE_ID)[0].pose_keypoints())
#esper_widget(stdlib.qs_to_result(Pose.objects.filter(pk=POSE_ID)))
for f in Pose.objects.annotate(video_id=F("frame__video__id")).distinct('video_id'):
    print(f.frame.video.name, f.frame.video.id)

mr mrs smith 123
star wars episode i the phantom menace 184
star wars episode ii attack of the clones 185
star wars episode iii revenge of the sith 186
star wars episode iv a new hope 187
star wars episode v the empire strikes back 188
star wars episode vi return of the jedi 189
the godfather 214
the godfather part ii 215
the godfather part iii 216
brooklyn 313
harry potter and the chamber of secrets 374
harry potter and the deathly hallows part 1 375
harry potter and the deathly hallows part 2 376
harry potter and the goblet of fire 377
harry potter and the halfblood prince 378
harry potter and the order of the phoenix 379
harry potter and the prisoner of azkaban 380
harry potter and the sorcerers stone 381
mr mrs smith 445
pillow talk 467
star wars the force awakens 519


In [10]:
for p in Pose.objects.filter(frame__video__id=380, frame__number=1296):
    print(p.pose_keypoints())
    print(pose_keypoints_to_shot_scale(p.pose_keypoints()))
print(np.arange(17))

[[ 0.15743743  0.78026277  0.2903161 ]
 [ 0.18593645  0.80215818  0.19284515]
 [ 0.16313151  0.88419163  0.09851257]
 [ 0.0788471   0.80221856  0.2018948 ]
 [ 0.          0.          0.        ]
 [ 0.20185547  0.73096406  0.17957135]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.14152636  0.79114497  0.36683878]
 [ 0.14943704  0.75013936  0.343256  ]
 [ 0.14035848  0.82679307  0.10837572]
 [ 0.15287919  0.72545677  0.1496032 ]]
ShotScale.M
[[ 0.60619158  0.27137747  0.58040065]
 [ 0.65288359  0.32067925  0.62806898]
 [ 0.67680913  0.31524536  0.54608732]
 [ 0.69502735  0.45474115  0.15907429]
 [ 0.          0.          0.        ]
 [ 0.6358009   0.32890078  0.50826317]
 [ 0.60959941  0.45198584  0.14767033]
 [ 0.       